# Recommendation Movie

## 1. Matrix Factorization, Content-based filtering

In [19]:
import re
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

movies = pd.read_csv('data/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
def clean_title(title):
    return re.sub(r'\(\d{4}\)', '', title).strip()
movies["title"] = movies["title"].apply(clean_title)
movies.head()

,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


In [21]:
movies["genres"] = movies["genres"].apply(lambda x: x.replace("|", " "))
movies["combine"] = movies["title"] + " " + movies["genres"]
movies.head()

,movieId,title,genres,combine
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,Toy Story Adventure Animation Children Comedy ...
1,2,Jumanji,Adventure Children Fantasy,Jumanji Adventure Children Fantasy
2,3,Grumpier Old Men,Comedy Romance,Grumpier Old Men Comedy Romance
3,4,Waiting to Exhale,Comedy Drama Romance,Waiting to Exhale Comedy Drama Romance
4,5,Father of the Bride Part II,Comedy,Father of the Bride Part II Comedy


In [22]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf = vectorizer.fit_transform(movies["combine"])
tfidf.shape

(62423, 154860)

In [23]:
title = "Old Men"
title = clean_title(title)
query_vector = vectorizer.transform([title])
query_vector.shape

(1, 154860)

In [24]:
cosine_sim = linear_kernel(query_vector, tfidf)
cosine_sim = cosine_sim.flatten()

related_movie_indices = cosine_sim.argsort()[:-6:-1]
result = movies.iloc[related_movie_indices]
result

,movieId,title,genres,combine
3354,3450,Grumpy Old Men,Comedy,Grumpy Old Men Comedy
32828,141820,Old Men: Robbers,Comedy,Old Men: Robbers Comedy
2,3,Grumpier Old Men,Comedy Romance,Grumpier Old Men Comedy Romance
11932,55820,No Country for Old Men,Crime Drama,No Country for Old Men Crime Drama
47016,173637,Seven Old Men and One Girl,Comedy,Seven Old Men and One Girl Comedy


## 2. Collaborative filtering based on user ratings

In [25]:
ratings = pd.read_csv("data/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [26]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [27]:
movie_id = 1
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] >= 4)]["userId"].unique()
similar_users

array([     3,      5,      8, ..., 162530, 162533, 162534], dtype=int64)